### SQL BASIC
#### What is SQL?
SQL (Structured Query Language) is a language used to manage data in relational databases.

Think of it like Excel formulas but for databases – it lets you create, read, update, and delete data in a structured way.

Documentation
Link to: sqlite3

Core SQL Commands
Here are the four essential SQL commands (often called CRUD operations):

### SELECT – Read data

In [1]:
SELECT * FROM coins; -- Gets all data from coins table

SELECT name, price FROM coins WHERE price > 100; — Gets specific columns with a condition

SyntaxError: invalid character '—' (U+2014) (1583808109.py, line 3)

### INSERT NEW RECORDS

In [ ]:
INSERT INTO coins VALUES ('Bitcoin', 50000); — Adds a new row

INSERT INTO coins (name, price) VALUES ('Ethereum', 3000); — Adds to specific columns

#### UPDATE – Modify Existing Records

In [ ]:
UPDATE coins SET price = 51000 WHERE name = 'Bitcoin'; 
-- Changes Bitcoin's price


#### DELETE – Remove Records

In [ ]:
DELETE FROM coins WHERE price < 10; 
-- Deletes all cheap coins


### Setting Up a Simple SQLite3 Database in Python
This tutorial creates a simple database to track cryptocurrencies.

Install pandas (if needed)

In [ ]:
import pandas as pd

In [ ]:
import sqlite3
import os

db_path = "crypto.db"

if os.path.exists(db_path):
    os.remove(db_path)
    print("Database file deleted.")
else:
    print("Database file does not exist.")


Database file does not exist.


In [ ]:
# Create or connect to a database file
conn = sqlite3.connect(db_path)

# Create a cursor object to execute SQL commands
cursor = conn.cursor()

# Create a table
cursor.execute("""
CREATE TABLE IF NOT EXISTS coins (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    symbol TEXT NOT NULL,
    price REAL,
    market_cap REAL,
    last_updated TIMESTAMP DEFAULT CURRENT_TIMESTAMP
)
""")

# Save (commit) the changes
conn.commit()
print("Table 'coins' created successfully.")


Table 'coins' created successfully.


Inserting Data

In [ ]:
# Insert single record
cursor.execute("""
INSERT INTO coins (name, symbol, price, market_cap)
VALUES ('Bitcoin', 'BTC', 50000, 950000000000)
""")

# Insert multiple records
coins = [
    ('Ethereum', 'ETH', 3000, 350000000000),
    ('Cardano', 'ADA', 1.5, 50000000000),
    ('Solana', 'SOL', 150, 50000000000)
]

cursor.executemany("""
INSERT INTO coins (name, symbol, price, market_cap)
VALUES (?, ?, ?, ?)
""", coins)

conn.commit()


### Querying Data

In [ ]:
# Get all coins
cursor.execute("SELECT * FROM coins")
all_coins = cursor.fetchall()  # Get all results as a list of tuples
print("All coins:", all_coins)

# Get one coin
cursor.execute("SELECT * FROM coins WHERE symbol = 'BTC'")
bitcoin = cursor.fetchone()  # Get just the first result
print("Bitcoin data:", bitcoin)

# Get specific columns with a condition
cursor.execute("SELECT name, price FROM coins WHERE price > 100")
expensive_coins = cursor.fetchall()
print("Expensive coins:", expensive_coins)


All coins: [(1, 'Bitcoin', 'BTC', 50000.0, 950000000000.0, '2025-08-20 15:29:47'), (2, 'Ethereum', 'ETH', 3000.0, 350000000000.0, '2025-08-20 15:29:47'), (3, 'Cardano', 'ADA', 1.5, 50000000000.0, '2025-08-20 15:29:47'), (4, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-20 15:29:47')]
Bitcoin data: (1, 'Bitcoin', 'BTC', 50000.0, 950000000000.0, '2025-08-20 15:29:47')
Expensive coins: [('Bitcoin', 50000.0), ('Ethereum', 3000.0), ('Solana', 150.0)]


### Updating and Deleting

In [ ]:
# Update Bitcoin's price
cursor.execute("UPDATE coins SET price = 51000 WHERE symbol = 'BTC'")
conn.commit()

# Delete cheap coins
cursor.execute("DELETE FROM coins WHERE price < 2")
conn.commit()


### Using Context Managers (Best Practice)

In [ ]:
# This automatically handles opening and closing the connection
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM coins")
    print(cursor.fetchall())
# No need to manually close - it's handled by the 'with' block


[(1, 'Bitcoin', 'BTC', 51000.0, 950000000000.0, '2025-08-20 15:29:47'), (2, 'Ethereum', 'ETH', 3000.0, 350000000000.0, '2025-08-20 15:29:47'), (4, 'Solana', 'SOL', 150.0, 50000000000.0, '2025-08-20 15:29:47')]


### Pandas + SQL Integration

In [ ]:
import pandas as pd

with sqlite3.connect(db_path) as conn:
    # Read SQL query directly into a DataFrame
    df = pd.read_sql("SELECT * FROM coins", conn)

print("DataFrame from SQL:")
print(df.head())


DataFrame from SQL:
   id      name symbol    price    market_cap         last_updated
0   1   Bitcoin    BTC  51000.0  9.500000e+11  2025-08-20 15:29:47
1   2  Ethereum    ETH   3000.0  3.500000e+11  2025-08-20 15:29:47
2   4    Solana    SOL    150.0  5.000000e+10  2025-08-20 15:29:47


### Writing Pandas to SQL

In [ ]:
# Create a new DataFrame
new_coins = pd.DataFrame({
    'name': ['Polkadot', 'Polygon'],
    'symbol': ['DOT', 'MATIC'],
    'price': [7.5, 0.75],
    'market_cap': [750000000, 500000000]
})

with sqlite3.connect(db_path) as conn:
    # Write the DataFrame to a new SQL table
    new_coins.to_sql("new_coins", conn, index=False, if_exists='replace')

    # Verify it worked
    df_new = pd.read_sql("SELECT * FROM new_coins", conn)
    print("\nNew coins table:")
    print(df_new)



New coins table:
       name symbol  price  market_cap
0  Polkadot    DOT   7.50   750000000
1   Polygon  MATIC   0.75   500000000


### Analyzing SQL Data With Pandas

In [ ]:
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("SELECT * FROM coins", conn)

# Now you can use all pandas operations
print("\nBasic statistics:")
print(df.describe())

print("\nAverage price:", df['price'].mean())
print("Total market cap:", df['market_cap'].sum())

# Filter with pandas
expensive = df[df['price'] > 100]
print("\nExpensive coins:")
print(expensive[['name', 'price']])



Basic statistics:
             id         price    market_cap
count  3.000000      3.000000  3.000000e+00
mean   2.333333  18050.000000  4.500000e+11
std    1.527525  28571.095534  4.582576e+11
min    1.000000    150.000000  5.000000e+10
25%    1.500000   1575.000000  2.000000e+11
50%    2.000000   3000.000000  3.500000e+11
75%    3.000000  27000.000000  6.500000e+11
max    4.000000  51000.000000  9.500000e+11

Average price: 18050.0
Total market cap: 1350000000000.0

Expensive coins:
       name    price
0   Bitcoin  51000.0
1  Ethereum   3000.0
2    Solana    150.0


### Joins & Relationships

#### Understanding Table Relationships

In real databases, data is often split across multiple related tables. For example: coins table: coin_id, name, symbol prices table: price_id, coin_id, price, date

#### Creating Related Tables

In [ ]:
with sqlite3.connect(db_path) as conn:
    cursor = conn.cursor()

    # Create coins table
    cursor.execute("DROP TABLE IF EXISTS coins")
    cursor.execute("""
    CREATE TABLE coins (
        coin_id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        symbol TEXT NOT NULL UNIQUE
    )
    """)

    # Create prices table with foreign key
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS prices (
        price_id INTEGER PRIMARY KEY AUTOINCREMENT,
        coin_id INTEGER NOT NULL,
        price REAL NOT NULL,
        date TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        FOREIGN KEY (coin_id) REFERENCES coins (coin_id)
    )
    """)
        # Insert some data
    cursor.execute("INSERT INTO coins (name, symbol) VALUES ('Bitcoin', 'BTC')")
    cursor.execute("INSERT INTO coins (name, symbol) VALUES ('Ethereum', 'ETH')")

    # Get the auto-generated IDs
    eth_id = cursor.lastrowid  # This would actually get ETH's ID - in real code you'd need to query for BTC's ID
    btc_id = eth_id - 1

    # Insert price history
    cursor.executemany("""
    INSERT INTO prices (coin_id, price)
    VALUES (?, ?)
    """, [
        (btc_id, 50000),
        (btc_id, 51000),
        (eth_id, 3000),
        (eth_id, 3100)
    ])

    conn.commit()


### PErforming Joins

In [ ]:
with sqlite3.connect(db_path) as conn:
    # Simple inner join
    df = pd.read_sql("""
    SELECT c.name, c.symbol, p.price, p.date
    FROM coins c
    INNER JOIN prices p ON c.coin_id = p.coin_id
    ORDER BY p.date DESC
    """, conn)

print("\nCoin prices with join:")
print(df)



Coin prices with join:
       name symbol    price                 date
0   Bitcoin    BTC  50000.0  2025-08-21 07:10:00
1   Bitcoin    BTC  51000.0  2025-08-21 07:10:00
2  Ethereum    ETH   3000.0  2025-08-21 07:10:00
3  Ethereum    ETH   3100.0  2025-08-21 07:10:00


In [ ]:
# Example of LEFT JOIN (get all coins even if they have no prices)
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
    SELECT c.name, c.symbol, p.price, p.date
    FROM coins c
    LEFT JOIN prices p ON c.coin_id = p.coin_id
    """, conn)

print("\nAll coins with prices (if available):")
print(df)



All coins with prices (if available):
       name symbol    price                 date
0   Bitcoin    BTC  50000.0  2025-08-21 07:10:00
1   Bitcoin    BTC  51000.0  2025-08-21 07:10:00
2  Ethereum    ETH   3000.0  2025-08-21 07:10:00
3  Ethereum    ETH   3100.0  2025-08-21 07:10:00


### BEST Practices 

In [ ]:
# Use context managers (with statement)
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
    SELECT *
    FROM coins c
    """, conn)


In [ ]:
# Close connection with commit (if not using context manager)
conn = sqlite3.connect(db_path)
df = pd.read_sql("""
SELECT *
FROM coins
""", conn)
conn.commit() #saving changes
conn.close() #closing connection


In [ ]:
# Index Frequent Columns to Improve Query Performance
# Lets say you are frequently searching by "symbol" or "name" in the db

conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Create an index on symbol
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_coins_symbol ON coins(symbol);
""")

# Create an index on name
cursor.execute("""
CREATE INDEX IF NOT EXISTS idx_coins_name ON coins(name);
""")

conn.commit()
conn.close()
print("Indexes created.")


Indexes created.


In [ ]:
with sqlite3.connect(db_path) as conn:
    df = pd.read_sql("""
        SELECT *
        FROM coins
        WHERE symbol = 'BTC'
    """, conn)

print(df)


   coin_id     name symbol
0        1  Bitcoin    BTC
